In [27]:
import os
import math
import numpy as np
import pandas as pd
import random
from IPython.display import display, HTML
from sklearn.preprocessing import OrdinalEncoder
import sys
sys.path.insert(1, '/home/ptr@itd.local/code/fairness_triangle/tools')  # Update this path as needed
from preprocessing import *
from sklearn.utils import resample



In [2]:
random.seed(42) # For reproducibility
sample_df = pd.read_csv("../datasets/2024_public_lar.csv", skiprows=lambda i: i > 0 and random.random() > 0.05)

/tmp/ipykernel_346767/3983548663.py:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv("../datasets/2024_public_lar.csv", skiprows=lambda i: i > 0 and random.random() > 0.05)


In [ ]:
# Find columns with >35% missing values
missing_ratio = sample_df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.35].index.tolist()
cols_to_drop = cols_to_drop + ['lei'] + ['applicant_age_above_62']
#print("Columns with >35% missing values:")
#print(cols_to_drop)

# Drop them from the DataFrame
sample_df_clean = sample_df.drop(columns=cols_to_drop)
print(f"Original shape: {sample_df.shape}")
print(f"New shape after drop: {sample_df_clean.shape}")

must_have_val = [
    'activity_year',
    'co_applicant_ethnicity_1',
    'co_applicant_race_1',
    'state_code',
    'county_code',
    'census_tract',
    'loan_amount',
    'property_value',
    'income',
    'debt_to_income_ratio',
    'applicant_credit_score_type',
    'action_taken',
    'aus_1',
    'applicant_race_1',
    'applicant_ethnicity_1',
    'applicant_sex'
]


sample_df_clean = sample_df_clean.dropna(subset=must_have_val)

Original shape: (611273, 99)
New shape after drop: (611273, 64)


In [38]:
#Make Y bar binary (m/w)
sample_df_clean['applicant_sex'].unique()
sample_df_clean = sample_df_clean[sample_df_clean['applicant_sex'].isin([1, 2])]
sample_df_clean['applicant_sex'] = sample_df_clean['applicant_sex'].replace({1: 0, 2: 1})

In [48]:
#Make Y binary (loan originated/not originated)
sample_df_clean['action_taken'] = sample_df_clean['action_taken'].apply(lambda x: 1 if x == 1 else 0)
print((sample_df_clean['action_taken'] == 1).mean() * 100)

49.57020663956639


In [ ]:
"""
df_1 = sample_df_clean[sample_df_clean['action_taken'] == 1]
df_0 = sample_df_clean[sample_df_clean['action_taken'] == 0]

# Downsample majority class to match minority class size
df_1_balanced = resample(df_1, replace=False, n_samples=len(df_0), random_state=42)

# Combine and shuffle
sample_df_clean = pd.concat([df_1_balanced, df_0]).sample(frac=1, random_state=42).reset_index(drop=True)
"""

In [ ]:
"""
df_1 = sample_df_clean[sample_df_clean['applicant_sex'] == 1]
df_0 = sample_df_clean[sample_df_clean['applicant_sex'] == 0]

# Downsample majority class to match minority class size
df_0_balanced = resample(df_0, replace=False, n_samples=len(df_1), random_state=42)

# Combine and shuffle
sample_df_clean = pd.concat([df_0_balanced, df_1]).sample(frac=1, random_state=42).reset_index(drop=True)
"""

In [42]:
percentage_ones = (sample_df_clean['applicant_sex'] == 1).mean() * 100
print(f"{percentage_ones:.2f}%")
percentage_ones = (sample_df_clean['action_taken'] == 1).mean() * 100
print(f"{percentage_ones:.2f}%")



50.00%
49.57%


In [ ]:
percentage_ones = (sample_df_clean['action_taken'] == 1).mean() * 100


In [7]:
# Convert loan_term to int and remove NaN
sample_df_clean['loan_term'] = pd.to_numeric(sample_df_clean['loan_term'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['loan_term'])
sample_df_clean['loan_term'] = sample_df_clean['loan_term'].astype(int)

In [8]:
# Convert loacombined_loan_to_value_ration_term to float and remove NaN
sample_df_clean['combined_loan_to_value_ratio'] = pd.to_numeric(sample_df_clean['combined_loan_to_value_ratio'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['combined_loan_to_value_ratio'])

In [9]:
# Convert interest_rate to float and remove NaN
sample_df_clean['interest_rate'] = pd.to_numeric(sample_df_clean['interest_rate'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['interest_rate'])

In [10]:
# Convert loan_term to int and remove NaN
sample_df_clean['property_value'] = pd.to_numeric(sample_df_clean['property_value'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['property_value'])
sample_df_clean['property_value'] = sample_df_clean['property_value'].astype(int)

In [50]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

49.57020663956639


In [11]:
# Encoding total_units ordinally and removing Nan
sample_df_clean['total_units'] = sample_df_clean['total_units'].astype(str).str.strip()

category_order = [['1', '2', '3', '4', '5-24', '25-49', '50-99', '>149']]


encoder = OrdinalEncoder(categories=category_order)
sample_df_clean['total_units'] = encoder.fit_transform(
    sample_df_clean[['total_units']]
).astype(int)

In [12]:
# Encoding applicant_age and co_applicant age and removing Nan
sample_df_clean['applicant_age'] = sample_df_clean['applicant_age'].replace('8888', np.nan)
sample_df_clean = sample_df_clean.dropna(subset=['applicant_age'])
age_order = [['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']]
encoder = OrdinalEncoder(categories=age_order)
sample_df_clean['applicant_age'] = encoder.fit_transform(sample_df_clean[['applicant_age']]).astype(int)


sample_df_clean['co_applicant_age'] = sample_df_clean['co_applicant_age'].replace(['8888', '9999'], np.nan)
sample_df_clean = sample_df_clean.dropna(subset=['co_applicant_age'])
age_order = [['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']]
encoder = OrdinalEncoder(categories=age_order)
sample_df_clean['co_applicant_age'] = encoder.fit_transform(sample_df_clean[['co_applicant_age']]).astype(int)


In [51]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

49.57020663956639


In [13]:
# Encoding state_code
sample_df_clean['state_code'] = sample_df_clean['state_code'].astype(str).str.strip()

# One-hot encode
state_dummies = pd.get_dummies(sample_df_clean['state_code'], prefix='state_code', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['state_code']), state_dummies], axis=1)


In [14]:
#Encode debt_to_income_ratio
sample_df_clean['debt_to_income_ratio'] = sample_df_clean['debt_to_income_ratio'].apply(parse_dti)

# Convert to integer if no NaNs remain, otherwise keep float
if sample_df_clean['debt_to_income_ratio'].isna().sum() == 0:
    sample_df_clean['debt_to_income_ratio'] = sample_df_clean['debt_to_income_ratio'].astype(int)

print(sample_df_clean['debt_to_income_ratio'].unique())



[42 25 20 55 48 33 46 44 41 36 40 49 43 45 38 47 37 39 60]


In [15]:
sample_df_clean['derived_race'] = sample_df_clean['derived_race'].astype(str).str.strip()
race_dummies = pd.get_dummies(sample_df_clean['derived_race'], prefix='derived_race', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_race']), race_dummies], axis=1)


In [16]:
sample_df_clean['derived_loan_product_type'] = sample_df_clean['derived_loan_product_type'].astype(str).str.strip()
loan_type_dummies = pd.get_dummies(sample_df_clean['derived_loan_product_type'], prefix='loan_product_type', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_loan_product_type']), loan_type_dummies], axis=1)


In [17]:
sample_df_clean['derived_ethnicity'] = sample_df_clean['derived_ethnicity'].astype(str).str.strip()
ethnicity_dummies = pd.get_dummies(sample_df_clean['derived_ethnicity'], prefix='derived_ethnicity', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_ethnicity']), ethnicity_dummies], axis=1)


In [18]:
sample_df_clean['derived_sex'] = sample_df_clean['derived_sex'].astype(str).str.strip()
sex_dummies = pd.get_dummies(sample_df_clean['derived_sex'], prefix='derived_sex', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_sex']), sex_dummies], axis=1)


In [19]:
sample_df_clean['derived_dwelling_category'] = sample_df_clean['derived_dwelling_category'].astype(str).str.strip()
dwelling_dummies = pd.get_dummies(sample_df_clean['derived_dwelling_category'], prefix='dwelling_category', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_dwelling_category']), dwelling_dummies], axis=1)


In [20]:
sample_df_clean = sample_df_clean.dropna(subset=['conforming_loan_limit'])
sample_df_clean['conforming_loan_limit'] = sample_df_clean['conforming_loan_limit'].map({'C': 1,'NC': 0})


In [56]:
Y = sample_df_clean['action_taken'].astype(int)
Y_sen = sample_df_clean['applicant_sex'].astype(int)
X =sample_df_clean.drop(columns=["action_taken", "applicant_sex"])


In [53]:
sample_df_clean['action_taken']

0         1
1         1
2         1
3         1
4         0
         ..
141691    1
141692    1
141693    0
141694    0
141695    0
Name: action_taken, Length: 141696, dtype: int64

In [52]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

49.57020663956639


In [57]:
pct_Y = (Y == 1).mean() * 100
pct_Y_sen_train = (Y_sen == 1).mean() * 100

print(f"Y_train {pct_Y:.2f}% are 1")
print(f"Y_test: {pct_Y_sen_train:.2f}% are 1")


Y_train 49.57% are 1
Y_test: 50.00% are 1
